# Analysis of sold cars in Estonia

Data is taken from official source - https://www.transpordiamet.ee/soidukitega-tehtud-toimingute-statistika

Period - 5 years.

Only new cars where used in this analysis.


In [73]:
%reload_ext autoreload
%autoreload 0

import os

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [74]:
from mnt_sum import get_summary, COLUMNS, COLUMN_SHORT_NAME, COLUMN_REG_DATE, COLUMN_CUSTOMER, PRIVATE_CUSTOMER, COLUMN_ENGINE_TYPE

df_o = pd.concat([get_summary(f"data/{year}") for year in range(2018, 2023)])
df_o = df_o[COLUMNS +[COLUMN_REG_DATE]]
df_o.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81625 entries, 42 to 1037
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Mark             81625 non-null  object 
 1   Mudel            81625 non-null  string 
 2   short name       81625 non-null  object 
 3   Mootori tüüp     81625 non-null  object 
 4   Mootori maht     81625 non-null  Int64  
 5   Mootori võimsus  81625 non-null  Float64
 6   Linn             81625 non-null  object 
 7   Tüüp (isik)      81625 non-null  object 
 8   Arv              81625 non-null  Int64  
 9   Värv             41233 non-null  string 
 10  Esm reg aasta    81625 non-null  Int16  
dtypes: Float64(1), Int16(1), Int64(2), object(5), string(2)
memory usage: 7.3+ MB


In [75]:
df_o.head(5)

,Mark,Mudel,short name,Mootori tüüp,Mootori maht,Mootori võimsus,Linn,Tüüp (isik),Arv,Värv,Esm reg aasta
42,ALFA ROMEO,GIULIA,ALFA ROMEO GIULIA,BENSIIN_KATALYSAATOR,1995,206.0,Määramata,JURIIDILINE,1,<NA>,2018
43,AUDI,A1 SPORTBACK,AUDI A1,BENSIIN_KATALYSAATOR,1395,110.0,Tallinn,FÜÜSILINE,1,<NA>,2018
44,AUDI,A4 AVANT,AUDI A4,BENSIIN_KATALYSAATOR,1984,185.0,Määramata,FÜÜSILINE,1,<NA>,2018
45,AUDI,A4 AVANT,AUDI A4,DIISEL,1968,140.0,Tallinn,JURIIDILINE,1,<NA>,2018
46,AUDI,A4 LIMOUSINE,AUDI A4,BENSIIN_KATALYSAATOR,1395,110.0,Tartu,FÜÜSILINE,1,<NA>,2018


In [76]:

df_models =  df_o.groupby(
        [COLUMN_SHORT_NAME], 
        as_index=False
    )['Arv'].sum().sort_values('Arv', ascending=False).reset_index(drop=True)
df_models.head()

,short name,Arv
0,TOYOTA RAV4,6443
1,SKODA OCTAVIA,4994
2,TOYOTA COROLLA,4334
3,RENAULT CLIO,3366
4,KIA SPORTAGE,2846


In [77]:
df_models_year =  df_o.groupby(
        [COLUMN_SHORT_NAME, COLUMN_REG_DATE], 
        as_index=False
    )['Arv'].sum().sort_values('Arv', ascending=False).reset_index(drop=True)

df_models_year.head()

,short name,Esm reg aasta,Arv
0,TOYOTA RAV4,2022,1526
1,TOYOTA RAV4,2021,1440
2,TOYOTA RAV4,2020,1400
3,TOYOTA RAV4,2019,1194
4,SKODA OCTAVIA,2021,1110


# Total overview

In [78]:
from matplotlib.colors import LinearSegmentedColormap


def sort_and_plot(_df, sortby=2022, limit=10):
    cm = LinearSegmentedColormap.from_list(
        name='ryg',
        colors=['red', 'yellow', 'green'],
    )

    df_group =  _df.groupby(
            [COLUMN_SHORT_NAME, COLUMN_REG_DATE], 
            as_index=False
        )['Arv'].sum().sort_values('Arv', ascending=False).reset_index(drop=True)


    df_h = df_group.pivot(
        index=COLUMN_SHORT_NAME, columns=COLUMN_REG_DATE, values='Arv'
    ).sort_values([sortby], ascending=False).astype('float64')

    df_h = df_h.head(limit)

    df_h.style.format(precision=0)
    df_colorized = df_h.style.background_gradient(cmap=cm).format(precision=0)

    return df_colorized

sort_and_plot(df_models_year)

Esm reg aasta,2018,2019,2020,2021,2022
short name,,,,,
TOYOTA RAV4,883,1194,1400,1440,1526
TOYOTA COROLLA,378,1091,977,922,966
SKODA OCTAVIA,911,1031,1054,1110,888
SKODA KODIAQ,462,675,536,480,691
KIA SPORTAGE,602,571,399,633,641
KIA CEED,533,511,403,664,512
TOYOTA YARIS CROSS,nan,nan,nan,49,498
TOYOTA C-HR,553,363,321,354,451
VOLKSWAGEN TIGUAN,398,449,288,337,369


# Private owners

In [79]:
df_private = df_o[df_o[COLUMN_CUSTOMER].apply(lambda x: PRIVATE_CUSTOMER in x)]

sort_and_plot(df_private)

Esm reg aasta,2018,2019,2020,2021,2022
short name,,,,,
TOYOTA RAV4,557,736,694,736,783
KIA SPORTAGE,350,287,202,405,350
SKODA KODIAQ,266,390,330,249,321
TOYOTA COROLLA,203,469,458,431,301
TOYOTA YARIS CROSS,nan,nan,nan,22,293
SKODA OCTAVIA,420,482,598,492,289
HONDA CR-V,235,176,151,190,210
KIA CEED,279,220,184,318,210
VOLKSWAGEN TIGUAN,262,318,195,221,201


# Private owners electric cars

In [82]:
df_private_e = df_private[df_private[COLUMN_ENGINE_TYPE] == 'ELEKTER']

sort_and_plot(df_private_e)

Esm reg aasta,2018,2019,2020,2021,2022
short name,,,,,
SKODA ENYAQ,nan,nan,nan,22,27
AUDI E-TRON,nan,nan,3,5,16
HYUNDAI IONIQ5,nan,nan,nan,4,11
TESLA MODEL 3,nan,nan,11,5,11
NISSAN LEAF,10,19,23,7,11
VOLVO XC40,nan,nan,nan,nan,11
DACIA SPRING,nan,nan,nan,nan,6
KIA NIRO,nan,nan,2,10,5
BMW I4,nan,nan,nan,nan,5
